In [184]:
# 모듈로딩
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchmetrics.classification import MulticlassF1Score
from torchinfo import summary


from torch.utils.data import Dataset,DataLoader

import pandas as pd
from sklearn.preprocessing import LabelEncoder,OneHotEncoder      # 타겟컬럼 수치화

In [185]:
# from urllib.request import urlretrieve

# # URL 경로 지정 
# TEST_URL  = 'https://media.githubusercontent.com/media/fpleoni/fashion_mnist/master/fashion-mnist_test.csv'
# TRAIN_URL = 'https://media.githubusercontent.com/media/fpleoni/fashion_mnist/master/fashion-mnist_train.csv'

# # 파일 이름 지정 
TEST_FILE = 'fasion_mnist_test.csv'
TRAIN_FILE = 'fasion_mnist_train.csv'

# # 데이터 파일로 저장
# urlretrieve(TRAIN_URL, TRAIN_FILE)
# urlretrieve(TEST_URL, TEST_FILE)

In [186]:
torch.manual_seed(1)

In [187]:
trainDF=pd.read_csv(TRAIN_FILE)
trainDF.head()


,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [188]:
trainDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 785 entries, label to pixel784
dtypes: int64(785)
memory usage: 359.3 MB


In [189]:
targetDF=trainDF[['label']]
featureDF=trainDF[trainDF.columns[1:]]

In [190]:
# # target 인코딩
encoder=LabelEncoder()
targetAr=encoder.fit_transform(targetDF)
# targetAr=targetDF.toarray()
targetAr=targetAr.reshape(-1,1)


c:\Users\KDP-30\anaconda3\envs\TORCH_38\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [191]:
targetTS=torch.FloatTensor(targetAr)
featureTS=torch.FloatTensor(featureDF.values)
print(f'feature : {featureTS.shape}, target : {targetTS.shape}')

feature : torch.Size([60000, 784]), target : torch.Size([60000, 1])


In [192]:
class CustomModel(nn.Module):
    
    def __init__(self,in_in,out_out,num_list=[]):
        super().__init__()
        self.num_list=num_list
        self.in_layer=nn.Linear(in_in,num_list[0] if len(num_list) else 5)
        self.h_layer=nn.ModuleList()
        for idx in range(len(num_list)-1):
            self.h_layer.append(nn.Linear(num_list[idx],num_list[idx+1]))
        self.out_layer=nn.Linear(num_list[-1] if len(num_list) else 5,out_out)

    def forward(self,X):
        X=F.relu(self.in_layer(X))
        for model in self.h_layer:
            X=model(X)
            X=F.relu(X)
        X=self.out_layer(X)
        return X


In [193]:
model=CustomModel(featureTS.shape[1],targetTS.shape[1],num_list=[10,100,100])

In [194]:
summary(model)

Layer (type:depth-idx)                   Param #
CustomModel                              --
├─Linear: 1-1                            7,850
├─ModuleList: 1-2                        --
│    └─Linear: 2-1                       5,500
│    └─Linear: 2-2                       50,100
├─Linear: 1-3                            101
Total params: 63,551
Trainable params: 63,551
Non-trainable params: 0

In [195]:
optimizer=optim.Adam(model.parameters(),lr=0.005)

In [196]:
def training(featureTS,targetTS,EPOCH,BATCH_SIZE=10):
    BATCH_CNT=featureTS.shape[0]//BATCH_SIZE
    loss_list=[]
    score_list=[]

    for epoch in range(EPOCH):
        total_loss=0
        total_score=0
        for i in range(BATCH_CNT):
            start=i * BATCH_SIZE
            end=start + BATCH_SIZE

            BS_feature=featureTS[start:end]
            BS_target=targetTS[start:end]

            pre_y=model(BS_feature)
            loss=nn.CrossEntropyLoss()(pre_y,BS_target.reshape(-1).long())
            total_loss+=loss.item()

            score=MulticlassF1Score(num_classes=10)(pre_y,BS_target)
            total_score+=score.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'[{epoch+1}/{EPOCH}] AVG LOSS {total_loss/BATCH_CNT},AVG SCORE {total_score/BATCH_CNT} ')

        loss_list.append(total_loss/BATCH_CNT)
        # score_list.append(total_score/BATCH_CNT)


    return loss_list,score_list

In [197]:
EPOCH=100
loss_list,score_list=training(featureTS,targetTS,EPOCH,BATCH_SIZE=10000)

RuntimeError: bincount only supports 1-d non-negative integral inputs.

In [ ]:
import matplotlib.pyplot as plt

# 마지막 loss 확인
num=100

fig,ax=plt.subplots(1,2,figsize=(10,5))
ax[0].plot(range(EPOCH-num,EPOCH),loss_list[-1*num:])
ax[0].set_xlabel('EPOCH')
ax[0].set_ylabel('loss') 

# ax[1].plot(range(EPOCH-num,EPOCH),score_list[-1*num:])
# ax[1].set_xlabel('EPOCH')
# ax[1].set_ylabel('score') 
plt.show()